In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import html5lib

In [2]:
standings_url = "https://fbref.com/de/wettbewerbe/20/Bundesliga-Statistiken"

In [3]:
data = requests.get(standings_url)

In [4]:
soup = BeautifulSoup(data.text)

In [5]:
standings_table = soup.select('table.stats_table') [0]

In [7]:
links = standings_table.find_all('a')

In [8]:
links = [l.get("href") for l in links]

In [9]:
links = [l for l in links if '/mannschaften/' in l]

In [10]:
links

['/de/mannschaften/c7a9f859/Bayer-Leverkusen-Statistiken',
 '/de/mannschaften/054efa67/Bayern-Munich-Statistiken',
 '/de/mannschaften/598bc722/Stuttgart-Statistiken',
 '/de/mannschaften/acbb6a5b/RB-Leipzig-Statistiken',
 '/de/mannschaften/add600ae/Dortmund-Statistiken',
 '/de/mannschaften/f0ac8ee6/Eintracht-Frankfurt-Statistiken',
 '/de/mannschaften/a486e511/Freiburg-Statistiken',
 '/de/mannschaften/0cdc4311/Augsburg-Statistiken',
 '/de/mannschaften/033ea6b8/Hoffenheim-Statistiken',
 '/de/mannschaften/18d9d2a7/Heidenheim-Statistiken',
 '/de/mannschaften/62add3bf/Werder-Bremen-Statistiken',
 '/de/mannschaften/32f3ee20/Monchengladbach-Statistiken',
 '/de/mannschaften/4eaa11d7/Wolfsburg-Statistiken',
 '/de/mannschaften/7a41008f/Union-Berlin-Statistiken',
 '/de/mannschaften/a224b06a/Mainz-05-Statistiken',
 '/de/mannschaften/b42c6323/Bochum-Statistiken',
 '/de/mannschaften/bc357bf7/Koln-Statistiken',
 '/de/mannschaften/6a6967fc/Darmstadt-98-Statistiken']

In [11]:
team_urls = [f"http://fbref.com{l}" for l in links]

In [12]:
team_urls

['http://fbref.com/de/mannschaften/c7a9f859/Bayer-Leverkusen-Statistiken',
 'http://fbref.com/de/mannschaften/054efa67/Bayern-Munich-Statistiken',
 'http://fbref.com/de/mannschaften/598bc722/Stuttgart-Statistiken',
 'http://fbref.com/de/mannschaften/acbb6a5b/RB-Leipzig-Statistiken',
 'http://fbref.com/de/mannschaften/add600ae/Dortmund-Statistiken',
 'http://fbref.com/de/mannschaften/f0ac8ee6/Eintracht-Frankfurt-Statistiken',
 'http://fbref.com/de/mannschaften/a486e511/Freiburg-Statistiken',
 'http://fbref.com/de/mannschaften/0cdc4311/Augsburg-Statistiken',
 'http://fbref.com/de/mannschaften/033ea6b8/Hoffenheim-Statistiken',
 'http://fbref.com/de/mannschaften/18d9d2a7/Heidenheim-Statistiken',
 'http://fbref.com/de/mannschaften/62add3bf/Werder-Bremen-Statistiken',
 'http://fbref.com/de/mannschaften/32f3ee20/Monchengladbach-Statistiken',
 'http://fbref.com/de/mannschaften/4eaa11d7/Wolfsburg-Statistiken',
 'http://fbref.com/de/mannschaften/7a41008f/Union-Berlin-Statistiken',
 'http://fbref

In [13]:
team_url = team_urls[0]

In [14]:
data = requests.get(team_url)

In [ ]:
data.text

In [16]:
matches = pd.read_html(data.text, match = "Ergebnisse & Spiele")

In [17]:
matches [0]

,Datum,Uhrzeit,Wett,Runde,Tag,Spielort,Ergebnis,Tf,Tk,Gegner,xG,xGA,Besitz,Zuschauerzahl,Kapitän,Aufstellung,Schiedsrichter,Spielbericht,Hinweise
0,12.08.2023,15:30,DFB-Pokal,Zweiunddreissigstelfinale,Sa.,Auswärts,S,8.0,0.0,FC Teutonia Ottensen,NaN,NaN,66.0,11.035,Lukáš Hrádecký,4-2-3-1,Tom Bauer,Spielbericht,NaN
1,19.08.2023,15:30,Bundesliga,Spielwoche 1,Sa.,Heim,S,3.0,2.0,RB Leipzig,1.4,1.6,44.0,29.464,Lukáš Hrádecký,3-4-3,Felix Brych,Spielbericht,NaN
2,26.08.2023,18:30,Bundesliga,Spielwoche 2,Sa.,Auswärts,S,3.0,0.0,M'Gladbach,2.7,0.9,60.0,54.042,Lukáš Hrádecký,3-4-3,Christian Dingert,Spielbericht,NaN
3,02.09.2023,15:30,Bundesliga,Spielwoche 3,Sa.,Heim,S,5.0,1.0,Darmstadt 98,2.8,0.4,72.0,29.653,Lukáš Hrádecký,3-4-3,Sven Jablonski,Spielbericht,NaN
4,15.09.2023,20:30,Bundesliga,Spielwoche 4,Fr.,Auswärts,U,2.0,2.0,Bayern München,2.1,2.1,49.0,75.000,Lukáš Hrádecký,3-4-3,Daniel Schlager,Spielbericht,NaN
5,21.09.2023,18:45,Europa Lg,Gruppenphase,Do.,Heim,S,4.0,0.0,se Häcken,3.0,0.4,63.0,25.402,Jonathan Tah,4-4-2,Manfredas Lukjančukas,Spielbericht,NaN
6,24.09.2023,15:30,Bundesliga,Spielwoche 5,So.,Heim,S,4.0,1.0,Heidenheim,3.4,0.5,71.0,29.557,Lukáš Hrádecký,3-4-3,Christian Dingert,Spielbericht,NaN
7,30.09.2023,15:30,Bundesliga,Spielwoche 6,Sa.,Auswärts,S,3.0,0.0,Mainz 05,1.1,0.5,55.0,33.305,Lukáš Hrádecký,3-4-3,Benjamin Brand,Spielbericht,NaN
8,05.10.2023,21:00,Europa Lg,Gruppenphase,Do.,Auswärts,S,2.0,1.0,no Molde,1.7,1.3,62.0,8.190,Jonathan Tah,3-4-3,Anastasios Papapetrou,Spielbericht,NaN
9,08.10.2023,15:30,Bundesliga,Spielwoche 7,So.,Heim,S,3.0,0.0,Köln,3.2,1.3,64.0,30.210,Lukáš Hrádecký,3-4-3,Felix Zwayer,Spielbericht,NaN


In [18]:
years = list(range(2024,2022,-1))

In [19]:
years

[2024, 2023]

In [20]:
all_matches = []

In [21]:
standings_url = "https://fbref.com/de/wettbewerbe/20/Bundesliga-Statistiken"

In [22]:
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table') [0]
    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/mannschaften/' in l]
    team_urls = [f"http://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com/{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Statistiken", "").replace("-"," ")
        
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match = "Ergebnisse & Spiele")[0]
        
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l ]
        data = requests.get(f"http://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match = "Torschüsse")[0]
        shooting.columns = shooting.columns.droplevel()
      
        try:
            team_data = matches.merge(shooting[["Datum","Sc", "SaT", "Entf.", "FS","Elf","VeElf"]], on="Datum", how= "outer")
        except ValueError:
            continue
        
        team_data = team_data[team_data["Wett"]== "Bundesliga"]
        team_data ["Saison"] = year
        team_data["Verein"] = team_name
        all_matches.append(team_data)
        time.sleep(1)
    
    

In [23]:
all_matches [0]

,Datum,Uhrzeit,Wett,Runde,Tag,Spielort,Ergebnis,Tf,Tk,Gegner,...,Spielbericht,Hinweise,Sc,SaT,Entf.,FS,Elf,VeElf,Saison,Verein
1,19.08.2023,15:30,Bundesliga,Spielwoche 1,Sa.,Heim,S,3.0,2.0,RB Leipzig,...,Spielbericht,NaN,11.0,7.0,190.0,0.0,0.0,0.0,2024,Bayer Leverkusen
2,26.08.2023,18:30,Bundesliga,Spielwoche 2,Sa.,Auswärts,S,3.0,0.0,M'Gladbach,...,Spielbericht,NaN,24.0,11.0,158.0,0.0,0.0,0.0,2024,Bayer Leverkusen
3,02.09.2023,15:30,Bundesliga,Spielwoche 3,Sa.,Heim,S,5.0,1.0,Darmstadt 98,...,Spielbericht,NaN,25.0,13.0,173.0,1.0,0.0,0.0,2024,Bayer Leverkusen
4,15.09.2023,20:30,Bundesliga,Spielwoche 4,Fr.,Auswärts,U,2.0,2.0,Bayern München,...,Spielbericht,NaN,12.0,4.0,207.0,1.0,1.0,1.0,2024,Bayer Leverkusen
6,24.09.2023,15:30,Bundesliga,Spielwoche 5,So.,Heim,S,4.0,1.0,Heidenheim,...,Spielbericht,NaN,20.0,9.0,167.0,0.0,1.0,1.0,2024,Bayer Leverkusen
7,30.09.2023,15:30,Bundesliga,Spielwoche 6,Sa.,Auswärts,S,3.0,0.0,Mainz 05,...,Spielbericht,NaN,6.0,3.0,206.0,2.0,0.0,0.0,2024,Bayer Leverkusen
9,08.10.2023,15:30,Bundesliga,Spielwoche 7,So.,Heim,S,3.0,0.0,Köln,...,Spielbericht,NaN,23.0,10.0,166.0,0.0,0.0,0.0,2024,Bayer Leverkusen
10,21.10.2023,15:30,Bundesliga,Spielwoche 8,Sa.,Auswärts,S,2.0,1.0,Wolfsburg,...,Spielbericht,NaN,13.0,7.0,141.0,0.0,0.0,0.0,2024,Bayer Leverkusen
12,29.10.2023,17:30,Bundesliga,Spielwoche 9,So.,Heim,S,2.0,1.0,Freiburg,...,Spielbericht,NaN,18.0,4.0,205.0,1.0,0.0,0.0,2024,Bayer Leverkusen
14,04.11.2023,15:30,Bundesliga,Spielwoche 10,Sa.,Auswärts,S,3.0,2.0,Hoffenheim,...,Spielbericht,NaN,9.0,5.0,178.0,0.0,0.0,0.0,2024,Bayer Leverkusen


In [24]:
match_df = pd.concat(all_matches)


In [25]:
match_df.columns = [c.lower() for c in match_df.columns]

In [26]:
match_df.to_csv("match.csv")

In [ ]:
import pickle

with open('match_df', 'wb') as f:
    pickle.dump(match_df, f)